<a href="https://colab.research.google.com/github/Jakaria6284/SentimentAnalysis-Using-Simple-RNN-/blob/master/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [ ]:
data =pd.read_csv('/content/test.csv',encoding='latin-1')

In [ ]:
data.head()

In [ ]:
data=data[['text','sentiment']]

In [ ]:
data.head()

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive


In [ ]:
data.isnull().sum()

,0
text,1281
sentiment,1281


In [ ]:
data = data[data['text'].apply(lambda x: isinstance(x, str))]

In [ ]:
import re

In [ ]:
def clean_text(text):
  text = re.sub(r'http\S+|www\S+', '', text)  # Remove URLs
  text = re.sub(r'@\w+', '', text)  # Remove mentions
  text = re.sub(r'[^a-zA-Z\s]', '', text)
   # Remove punctuation and numbers
  return text.lower()



In [ ]:
data['text']=data['text'].apply(clean_text)

In [ ]:
data.head()

,text,sentiment
0,last session of the day,neutral
1,shanghai is also really exciting precisely s...,positive
2,recession hit veronique branquinho she has to ...,negative
3,happy bday,positive
4,i like it,positive


In [ ]:
max_words=10000
max_len=100

In [ ]:
tokenizer=Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['text'])
X=tokenizer.texts_to_sequences(data['text'])
X=pad_sequences(X,maxlen=max_len)


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
data['sentiment']=label_encoder.fit_transform(data['sentiment'])
print(data['sentiment'].value_counts())  # See counts of each sentiment class
print(label_encoder.inverse_transform([0, 1, 2]))  # Get corresponding text labels



sentiment
1    1430
2    1103
0    1001
Name: count, dtype: int64
['negative' 'neutral' 'positive']


In [ ]:
y=to_categorical(data['sentiment'],num_classes=3)
y

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

# 1. Build Simple RNN model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(3, activation='softmax'))  # Output layer with 2 classes (positive/negative)

# 2. Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 3. Train the model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# 4. Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 8s 114ms/step - accuracy: 0.3605 - loss: 1.1034 - val_accuracy: 0.4116 - val_loss: 1.0755
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - accuracy: 0.7546 - loss: 0.8232 - val_accuracy: 0.4286 - val_loss: 1.0915
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 101ms/step - accuracy: 0.9241 - loss: 0.3710 - val_accuracy: 0.4399 - val_loss: 1.2649
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.9853 - loss: 0.1157 - val_accuracy: 0.4441 - val_loss: 1.3735
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - accuracy: 0.9942 - loss: 0.0532 - val_accuracy: 0.4470 - val_loss: 1.5414
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 111ms/step - accuracy: 0.9970 - loss: 0.0259 - val_accuracy: 0.4512 - val_loss: 1.6265
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 0.9974 - loss: 0.0279 - val_accuracy: 0.4427 - val_loss: 1.7338
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.9993 - loss: 0.0168 - val_accuracy: 0.4300 

In [ ]:
def predict_sentiment(text):
    seq = tokenizer.texts_to_sequences([text])  # Convert text to sequence
    padded_seq = pad_sequences(seq, maxlen=max_len)  # Pad sequence
    pred = model.predict(padded_seq)  # Predict sentiment
    return ["neutral", "negative", "positive"][np.argmax(pred)]  # Get sentiment label

# Example usage
print(predict_sentiment("i fuck you"))  # Output: positive
print(predict_sentiment("i hate you"))    # Output: negative


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
neutral


In [ ]:
!pip install pyngrok nest_asyncio fastapi uvicorn tensorflow



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 1.4 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok
!ngrok authtoken #your ngrok auth key


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import uvicorn
import nest_asyncio
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Apply nest_asyncio to allow running FastAPI in Colab
nest_asyncio.apply()

app = FastAPI()

# Important: Add CORS middleware to allow requests from your Flutter app
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # For development only - restrict this in production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load trained model and tokenizer
# Make sure these are defined before this point in your Colab notebook
# model = ...  # Your trained model
# tokenizer = ...  # Your fitted tokenizer
max_len = 100

# Function to predict sentiment
def predict_sentiment(text):
    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=max_len)
    pred = model.predict(padded_seq)
    return ["neutral", "negative", "positive"][np.argmax(pred)]

# FastAPI route
@app.get("/predict/")
def get_sentiment(text: str):
    sentiment = predict_sentiment(text)
    return {"sentiment": sentiment}

# Create a new ngrok tunnel
from pyngrok import ngrok

# Start FastAPI with Uvicorn
port = 8000  # Define port

# Expose API using Ngrok
public_url = ngrok.connect(port).public_url
print(f"Public URL: {public_url}")  # COPY THIS URL TO YOUR FLUTTER APP

# Run the FastAPI server
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=port)

INFO:     Started server process [1361]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://8b1c-35-243-143-244.ngrok-free.app
INFO:     203.76.220.232:0 - "OPTIONS /predict/?text=hello%20how%20are%20you HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
INFO:     203.76.220.232:0 - "GET /predict/?text=hello%20how%20are%20you HTTP/1.1" 200 OK
INFO:     203.76.220.232:0 - "OPTIONS /predict/?text=i%20wanna%20kill%20you HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
INFO:     203.76.220.232:0 - "GET /predict/?text=i%20wanna%20kill%20you HTTP/1.1" 200 OK
INFO:     203.76.220.232:0 - "OPTIONS /predict/?text=your%20product%20is%20good HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
INFO:     203.76.220.232:0 - "GET /predict/?text=your%20product%20is%20good HTTP/1.1" 200 OK
INFO:     203.76.220.232:0 - "OPTIONS /predict/?text=Suzuki%20Swift%20experience%20%E2%80%93%20Bought%20the%20car%20in%202022%2C%20with%2090%2C000%20km%20mileage%20at%20that%20time.%20It%20is%20now%20with%20124k%20km%20in%20Feb%202025.%20Real%20life%20combined%20fuel%20effic

In [ ]:
from pyngrok import ngrok

# Start FastAPI with Uvicorn
port = 8000  # Define port

# Expose API using Ngrok
public_url = ngrok.connect(port).public_url
print(f"Public URL: {public_url}")

# Run the FastAPI server
uvicorn.run(app, host="0.0.0.0", port=port)


Public URL: https://6a7b-34-138-164-71.ngrok-free.app


INFO:     Started server process [210]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     203.76.220.235:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     203.76.220.235:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     203.76.220.235:0 - "GET /docs HTTP/1.1" 200 OK


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-12' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

INFO:     203.76.220.235:0 - "GET /openapi.json HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
INFO:     203.76.220.235:0 - "GET /predict/?text=i%20love%20you HTTP/1.1" 200 OK
INFO:     203.76.220.232:0 - "GET / HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [210]
